<a href="https://colab.research.google.com/github/anhvt00/PIPR/blob/master/models/PIPR_LGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook use for tunning model using embeddings file and language model embedder

### Check GPU hardware

In [1]:
!nvidia-smi

Fri Feb  4 03:45:06 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Download embedding files

In [2]:
# Download file seq2tensor.py for converting protein sequences to tensors
!wget https://raw.githubusercontent.com/anhvt00/PIPR/master/embeddings/seq2tensor.py

# Download file ac5_aph.txt for ac5_aph embedding 
!wget https://raw.githubusercontent.com/anhvt00/PIPR/master/embeddings/ac5_aph.txt

### Download interaction pairs and dictionary files
# Download dictionary file (id: sequence)
# Download dictionary file (id: sequence)
# !wget https://raw.githubusercontent.com/anhvt00/PIPR/master/data/Tunning-architecture-dataset/yeast_pairs.tsv
!wget https://raw.githubusercontent.com/anhvt00/PIPR/master/data/Golden-tunning-datasets/S.cerevisae/yeast_pairs.tsv
!wget https://raw.githubusercontent.com/anhvt00/PIPR/master/data/Golden-tunning-datasets/H.pylori/hp_pairs.tsv
!wget https://raw.githubusercontent.com/anhvt00/PIPR/master/data/Golden-tunning-datasets/H.pylori/hp_dict.tsv
# Download pairs of proteins with labels file
# !wget https://raw.githubusercontent.com/anhvt00/PIPR/master/data/Tunning-architecture-dataset/yeast_dictionary.tsv
!wget https://raw.githubusercontent.com/anhvt00/PIPR/master/data/Golden-tunning-datasets/S.cerevisae/yeast_dictionary.tsv

--2022-02-04 03:45:07--  https://raw.githubusercontent.com/anhvt00/PIPR/master/embeddings/seq2tensor.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1104 (1.1K) [text/plain]
Saving to: ‘seq2tensor.py’

seq2tensor.py       100%[===================>]   1.08K  --.-KB/s    in 0s      

2022-02-04 03:45:07 (54.4 MB/s) - ‘seq2tensor.py’ saved [1104/1104]

--2022-02-04 03:45:07--  https://raw.githubusercontent.com/anhvt00/PIPR/master/embeddings/ac5_aph.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2335 (2.3K) [text/plain]
Saving

### Import libraries 

In [3]:
# Libraries for system and debug
import sys
import pdb
import os
from datetime import datetime

# Class for converting sequences to tensors
from seq2tensor import s2t


# Libraries for neural network training
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GRU, LSTM, Bidirectional, Input, Conv1D, Conv2D
from tensorflow.keras.layers import Add, Flatten, subtract, multiply, concatenate
from tensorflow.keras.layers import MaxPooling1D, AveragePooling1D, GlobalAveragePooling1D, MaxPooling2D
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.utils import Sequence
from tensorflow.keras import mixed_precision
from tensorflow import keras
from tensorboard.plugins.hparams import api as hp
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.layers import Activation
from keras.callbacks import ModelCheckpoint
from tensorflow.keras import regularizers
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef,accuracy_score, precision_score,recall_score
from sklearn.manifold import TSNE
# Import accessory modules
import numpy as np
import h5py
import gc
from tqdm import tqdm
import math
import pandas as pd

# For tensorboard extension
!pip install -U tensorboard_plugin_profile


!pip install lightgbm
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import lightgbm as lgb

     |████████████████████████████████| 1.1 MB 7.8 MB/s 


### Set CUDA environment variables

In [4]:
### Setting RAM GPU for training growth 
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)





# ============================================
# Optimisation Flags - Do not remove
# ============================================

# Disables caching (when set to 1) or enables caching (when set to 0) for just-in-time-compilation. When disabled,
# no binary code is added to or retrieved from the cache.
os.environ['CUDA_CACHE_DISABLE'] = '0' # orig is 0

# When set to 1, forces the device driver to ignore any binary code embedded in an application 
# (see Application Compatibility) and to just-in-time compile embedded PTX code instead.
# If a kernel does not have embedded PTX code, it will fail to load. This environment variable can be used to
# validate that PTX code is embedded in an application and that its just-in-time compilation works as expected to guarantee application 
# forward compatibility with future architectures.
os.environ['CUDA_FORCE_PTX_JIT'] = '1'# no orig


os.environ['HOROVOD_GPU_ALLREDUCE'] = 'NCCL'

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

os.environ['TF_GPU_THREAD_MODE'] = 'gpu_private'
os.environ['TF_GPU_THREAD_COUNT']='1'

os.environ['TF_USE_CUDNN_BATCHNORM_SPATIAL_PERSISTENT'] = '1'

os.environ['TF_ADJUST_HUE_FUSED'] = '1'
os.environ['TF_ADJUST_SATURATION_FUSED'] = '1'
os.environ['TF_ENABLE_WINOGRAD_NONFUSED'] = '1'

os.environ['TF_SYNC_ON_FINISH'] = '0'
os.environ['TF_AUTOTUNE_THRESHOLD'] = '2'
os.environ['TF_DISABLE_NVTX_RANGES'] = '1'
os.environ["TF_ENABLE_AUTO_MIXED_PRECISION_GRAPH_REWRITE"] = "1"



# =================================================
# mixed_precision.set_global_policy('mixed_float16')

1 Physical GPUs, 1 Logical GPUs


### Hyperparameter set by default



In [5]:
# Default hyperparameters
CONV_HIDDEN_DIM = 50
RNN_HIDDEN = 50
N_EPOCHS = 80
HIDDEN_DIM=50
BATCH_SIZE = 32
DTYPE='float32'
LEARNING_RATE=.001
EPSILON=1e-6
adam = Adam(learning_rate=LEARNING_RATE, amsgrad=True, epsilon=EPSILON)
MAX_DATASET_SIZE = 11187
DATASET_SIZE = MAX_DATASET_SIZE
KERNEL_SIZE = 3
POOLING_KERNEL = 3
seq_size=3500
dim = 1024
# 1 for language model embedding
flags_embedding = 0
# 1 for loading from drive
available_data = 0
augment_data = True



### Load the embeddings from drive

In [6]:
if available_data == 1:
    from google.colab import drive
    drive.mount('/content/drive')

    seq_tensor = np.load('/content/drive/MyDrive/seq_tensor.npy', allow_pickle=True)
    # Load contextual embeddings here
    # seq_tensor = np.load('/content/drive/MyDrive/Embeddings_Guo/15_amino_Word2VecEmbedder.npy', allow_pickle=True)

    class_labels = np.load('/content/drive/MyDrive/class_labels.npy', allow_pickle=True)
    seq_index1 = np.load('/content/drive/MyDrive/seq_index1.npy', allow_pickle=True)
    seq_index2 = np.load('/content/drive/MyDrive/seq_index2.npy', allow_pickle=True)
    # seq_tensor_physicochemical= np.load('/content/drive/MyDrive/physicochemical.npy', allow_pickle=True)

### Conjoint Triad

In [7]:
# # https://sci-hub.se/https://doi.org/10.3390/ijms18112373
# # After that, the conjoint triad method is introduced to extract the sequence information, which includes the properties of one amino acid and its vicinal amino acids and regards any 
# # three continuous amino acids as a unit. Firstly, they replaced each amino acid in the protein sequence by the index depending on its grouping. For instance, protein sequence
# # “VCCPPVCVVCPPVCVPVPPCCV” is replaced by 0112201001220102022110. Then, binary space (V, F) stands for a protein sequence. Here, V is the vector sp… acids grouped into seven classes, the size V 
# # should be 7 × 7 × 7; therefore, i = 0, 1, · · · , 342. The detailed definition and description is shown in Figure 4. Clearly, each protein has a corresponding F vector. Nevertheless, the value
# # of fi relates to the length of amino acid sequence. A longer amino acid sequence generally have a larger value of fi, which complicates the comparison between two heterogeneous proteins. 
# # As such they employed the normalization to solve this problem as follows: di = (fi - min fi) / max fi for i = 1 to 343, where the value of di is normalized in the range [0, 1]. fi is the
# # frequency of conjoint triad unit vi appearing in the protein sequence. Finally, they connected the vector spaces of two proteins to present the interaction features. Thus, a 686-dimensional 
# # vector (343 for each protein) is generated for each pair of proteins.

# # original paper: https://www.pnas.org/content/104/11/4337 (shen et al., 2007)
# ct_encoding = {
#     "A": 0,
#     "G": 0,
#     "V": 0,
#     "C": 1,
#     "F": 2,
#     "I": 2,
#     "L": 2,
#     "P": 2,
#     "M": 3,
#     "S": 3,
#     "T": 3,
#     "Y": 3,
#     "H": 4,
#     "N": 4,
#     "Q": 4,
#     "W": 4,
#     "K": 5,
#     "R": 5,
#     "D": 6,
#     "E": 6,
# }

# f = open('encode_ct.txt', 'w')
# for aa, value in ct_encoding.items():
#   f.write(str(aa)+'\t'+str(value)+'\n')

# f.close()

In [8]:
"""Implementation of CT coding method
"""

__all__ = ['ct_code_of']

# AAC: Classification of amino acids.
AAC = {
    '1': ['A', 'G', 'V'],
    '2': ['I', 'L', 'F', 'P'],
    '3': ['Y', 'M', 'T', 'S'],
    '4': ['H', 'N', 'Q', 'W'],
    '5': ['R', 'K'],
    '6': ['D', 'E'],
    '7': ['C']
}

# AAC_R: Reverse of AAC.
AAC_R = {}
for C, AAS in AAC.items():
    for AA in AAS:
        AAC_R[AA] = C

def classification_of(AA):
    """Get classification of amino acids."""
    return AAC_R[AA]

def classification_sequence_of(PS):
    """Make classification sequence from protein sequence."""
    CS = ''
    for I, CH in enumerate(PS):
        CS = CS + classification_of(CH)
    return CS

def ct_code_of(PS):
    """Get CT Code of protein sequence."""
    CT_Code = [0]*343
    CS = classification_sequence_of(PS)
    for I in range(len(CS)-2):
        SubCS = CS[I:I+3]
        CT_Code_Index = int(SubCS[0]) + (int(SubCS[1])-1)*7 + (int(SubCS[2])-1)*7*7
        CT_Code[CT_Code_Index-1] = CT_Code[CT_Code_Index-1] + 1
    SUM = sum(CT_Code)
    CT_Code = [N*1.0/SUM for N in CT_Code]
    # Normalizing CT_Code
    # MIN_CODE = min(CT_Code)
    # MAX_CODE = max(CT_Code)
    # CT_Code = [(N-MIN_CODE)*1.0/(MAX_CODE-MIN_CODE) for N in CT_Code]
    return CT_Code

if __name__=="__main__":
    for I, Frequency in enumerate(ct_code_of('MREIVHIQAG')):
        if Frequency>0 :
            print(I+1, Frequency)

4 0.125
13 0.125
23 0.125
71 0.125
89 0.125
149 0.125
158 0.125
276 0.125


### Local Descriptor

In [9]:
"""Implementation of LD coding method
"""

__all__ = ['ld_code_of']

# AAC: Classification of amino acids.
AAC = {
    # '0': ['X'],
    '1': ['A', 'G', 'V'],
    '2': ['I', 'L', 'F', 'P'],
    '3': ['Y', 'M', 'T', 'S'],
    '4': ['H', 'N', 'Q', 'W'],
    '5': ['R', 'K'],
    '6': ['D', 'E'],
    '7': ['C']
}

# AAC_R: Reverse of AAC.
AAC_R = {}
for C, AAS in AAC.items():
    for AA in AAS:
        AAC_R[AA] = C

def classification_of(AA):
    """Get classification of amino acids."""
    return AAC_R[AA]

def classification_sequence_of(PS):
    """Make classification sequence from protein sequence."""
    CS = ''
    for I, CH in enumerate(PS):
        if CH == 'X':
          CS = CS
        else:
          CS = CS + classification_of(CH)
    return CS

def ld_info_of(CS):
    L = len(CS)
    C = {}
    T = {}
    for I, CH in enumerate(CS):
        if CH not in C:
            C[CH] = []
        C[CH].append(I+1)
        if I > 0:
            PCH = CS[I-1]
            if PCH != CH:
                if int(PCH)<int(CH):
                    TIndex = PCH + CH
                else:
                    TIndex = CH + PCH
                if TIndex not in T:
                    T[TIndex] = 0
                T[TIndex] = T[TIndex]+1
    return L, C, T

def ld_code_of_0(CS):
    RC = [0]*7
    RT = [0]*21
    RD = [0]*35
    L, C, T = ld_info_of(CS)
    for Class, Indexs in C.items():
        Len = len(Indexs)
        RC[int(Class)-1]=Len*1.0/L
        Residues = [1, int(Len*0.25), int(Len*0.5), int(Len*0.75), Len]
        # Residues = list(map(lambda x:x*1.0/L, Residues))
        Residues = list(map(lambda x:Indexs[x-1]*1.0/L, Residues))
        RD[(int(Class)-1)*5:int(Class)*5] = Residues
    for Trans, Frequency in T.items():
        PI, I = int(Trans[0])-1, int(Trans[1])-1
        Index = int((21-(6-PI)*(6-PI+1)/2)+(I-PI-1))
        RT[Index] = Frequency*1.0/(L-1)
    # return RC, RT, RD
    return RC+RT+RD

def ld_code_of(PS):
    """Get LD Code of protein sequence."""
    CS = classification_sequence_of(PS)
    L = len(CS)
    A = ld_code_of_0(CS[          0:int(L*0.25)])
    B = ld_code_of_0(CS[int(L*0.25):int(L*0.50)])
    C = ld_code_of_0(CS[int(L*0.50):int(L*0.75)])
    D = ld_code_of_0(CS[int(L*0.75):L          ])
    E = ld_code_of_0(CS[          0:int(L*0.50)])
    F = ld_code_of_0(CS[int(L*0.50):L          ])
    G = ld_code_of_0(CS[int(L*0.25):int(L*0.75)])
    H = ld_code_of_0(CS[          0:int(L*0.75)])
    I = ld_code_of_0(CS[int(L*0.25):L          ])
    J = ld_code_of_0(CS[int(L*0.125):int(L*0.875)])
    return A+B+C+D+E+F+G+H+I+J

if __name__=="__main__":
    PS = 'VCCPPVCVVCPPVCVPVPPCCV'
    print('PS=', PS)
    CS = classification_sequence_of(PS)
    print('CS=', CS)
    LD_INFO = ld_info_of(CS)
    print('LD_INFO=', LD_INFO)
    LD_CODE = ld_code_of_0(CS)
    print('LD_CODE=', LD_CODE)
    LD_CODE = ld_code_of(PS)
    print('LD_CODE=', LD_CODE)

PS= VCCPPVCVVCPPVCVPVPPCCV
CS= 1772217117221712122771
LD_INFO= (22, {'1': [1, 6, 8, 9, 13, 15, 17, 22], '7': [2, 3, 7, 10, 14, 20, 21], '2': [4, 5, 11, 12, 16, 18, 19]}, {'17': 7, '27': 3, '12': 5})
LD_CODE= [0.36363636363636365, 0.3181818181818182, 0, 0, 0, 0, 0.3181818181818182, 0.23809523809523808, 0, 0, 0, 0, 0.3333333333333333, 0, 0, 0, 0, 0.14285714285714285, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.045454545454545456, 0.2727272727272727, 0.4090909090909091, 0.6818181818181818, 1.0, 0.18181818181818182, 0.18181818181818182, 0.5, 0.7272727272727273, 0.8636363636363636, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.09090909090909091, 0.09090909090909091, 0.3181818181818182, 0.6363636363636364, 0.9545454545454546]
LD_CODE= [0.2, 0.4, 0, 0, 0, 0, 0.4, 0, 0, 0, 0, 0, 0.25, 0, 0, 0, 0, 0.25, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.2, 0.2, 0.2, 0.2, 0.2, 0.8, 1.0, 0.8, 0.8, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.4, 0.6, 0.4, 0.4, 0.6, 0.5, 0.16666666666666666,

### Auto Covariance

In [10]:
# alphabet = ["A", "C", "D", "E", "F", "G", "H", "I", "K", "L", "M", "N", "P", "Q", "R", "S", "T", "V", "W", "Y"]
# chem_props = np.array([0.62, -0.5,0.007187,8.1,0.046,1.181,27.5,0.29,-1,-0.03661,5.5,0.128,1.461,44.6,-0.9,3,-0.02382,13,0.105,1.587,40,-0.74,3,0.006802,12.3,0.151,1.862,62,1.19,-2.5,0.037552,5.2,0.29,2.228,115.5,0.48,0,0.179052,9,0,0.881,0,-0.4,-0.5,-0.01069,10.4,0.23,2.025,79,1.38,-1.8,0.021631,5.2,0.186,1.81,93.5,-1.5,3,0.017708,11.3,0.219,2.258,100,1.06,-1.8,0.051672,4.9,0.186,1.931,93.5,0.64,-1.3,0.002683,5.7,0.221,2.034,94.1,-0.78,2,0.005392,11.6,0.134,1.655,58.7,0.12,0,0.239531,8,0.131,1.468,41.9,-0.85,0.2,0.049211,10.5,0.18,1.932,80.7,-2.53,3,0.043587,10.5,0.291,2.56,105,-0.18,0.3,0.004627,9.2,0.062,1.298,29.3,-0.05,-0.4,0.003352,8.6,0.108,1.525,51.3,1.08,-1.5,0.057004,5.9,0.14,1.645,71.5,0.81,-3.4,0.037977,5.4,0.409,2.663,145.5,0.26,-2.3,117.3,6.2,0.298,2.368,0.023599])
# chem_props = chem_props.reshape(len(alphabet),7)

# # a = np.asarray([ [1,2,3], [4,5,6], [7,8,9] ])
# # for i in range(7):
# import pandas as pd 
# df = pd.DataFrame(chem_props)
# df = df.rename(index={0: "A", 1: "C", 2: "D", 3: "E", 4: "F", 5: "G", 6: "H", 7: "I", 8: "K", 9: "L", 10: "M", 11: "N", 12: "P", 13:"Q", 14:"R", 15: "S", 16:"T", 17: "V", 18: "W", 19: "Y"})
# df.to_csv("encode_ac.txt", sep='\t', header=None)
# # df

In [11]:
'''Implementation of AC coding method
'''

__all__ = ['ac_code_of']

# PCPNS: Physicochemical property names
PCPNS = ['H1', 'H2', 'NCI', 'P1', 'P2', 'SASA', 'V']

# AAPCPVS: Physicochemical property values of amino acid
AAPCPVS = {
    'A': { 'H1': 0.62, 'H2':-0.5, 'NCI': 0.007187, 'P1': 8.1, 'P2':0.046, 'SASA':1.181, 'V': 27.5 },
    'C': { 'H1': 0.29, 'H2':-1.0, 'NCI':-0.036610, 'P1': 5.5, 'P2':0.128, 'SASA':1.461, 'V': 44.6 },
    'D': { 'H1':-0.90, 'H2': 3.0, 'NCI':-0.023820, 'P1':13.0, 'P2':0.105, 'SASA':1.587, 'V': 40.0 },
    'E': { 'H1': 0.74, 'H2': 3.0, 'NCI': 0.006802, 'P1':12.3, 'P2':0.151, 'SASA':1.862, 'V': 62.0 },
    'F': { 'H1': 1.19, 'H2':-2.5, 'NCI': 0.037552, 'P1': 5.2, 'P2':0.290, 'SASA':2.228, 'V':115.5 },
    'G': { 'H1': 0.48, 'H2': 0.0, 'NCI': 0.179052, 'P1': 9.0, 'P2':0.000, 'SASA':0.881, 'V':  0.0 },
    'H': { 'H1':-0.40, 'H2':-0.5, 'NCI':-0.010690, 'P1':10.4, 'P2':0.230, 'SASA':2.025, 'V': 79.0 },
    'I': { 'H1': 1.38, 'H2':-1.8, 'NCI': 0.021631, 'P1': 5.2, 'P2':0.186, 'SASA':1.810, 'V': 93.5 },
    'K': { 'H1':-1.50, 'H2': 3.0, 'NCI': 0.017708, 'P1':11.3, 'P2':0.219, 'SASA':2.258, 'V':100.0 },
    'L': { 'H1': 1.06, 'H2':-1.8, 'NCI': 0.051672, 'P1': 4.9, 'P2':0.186, 'SASA':1.931, 'V': 93.5 },
    'M': { 'H1': 0.64, 'H2':-1.3, 'NCI': 0.002683, 'P1': 5.7, 'P2':0.221, 'SASA':2.034, 'V': 94.1 },
    'N': { 'H1':-0.78, 'H2': 2.0, 'NCI': 0.005392, 'P1':11.6, 'P2':0.134, 'SASA':1.655, 'V': 58.7 },
    'P': { 'H1': 0.12, 'H2': 0.0, 'NCI': 0.239531, 'P1': 8.0, 'P2':0.131, 'SASA':1.468, 'V': 41.9 },
    'Q': { 'H1':-0.85, 'H2': 0.2, 'NCI': 0.049211, 'P1':10.5, 'P2':0.180, 'SASA':1.932, 'V': 80.7 },
    'R': { 'H1':-2.53, 'H2': 3.0, 'NCI': 0.043587, 'P1':10.5, 'P2':0.291, 'SASA':2.560, 'V':105.0 },
    'S': { 'H1':-0.18, 'H2': 0.3, 'NCI': 0.004627, 'P1': 9.2, 'P2':0.062, 'SASA':1.298, 'V': 29.3 },
    'T': { 'H1':-0.05, 'H2':-0.4, 'NCI': 0.003352, 'P1': 8.6, 'P2':0.108, 'SASA':1.525, 'V': 51.3 },
    'V': { 'H1': 1.08, 'H2':-1.5, 'NCI': 0.057004, 'P1': 5.9, 'P2':0.140, 'SASA':1.645, 'V': 71.5 },
    'W': { 'H1': 0.81, 'H2':-3.4, 'NCI': 0.037977, 'P1': 5.4, 'P2':0.409, 'SASA':2.663, 'V':145.5 },
    'Y': { 'H1': 0.26, 'H2':-2.3, 'NCI': 117.3000, 'P1': 6.2, 'P2':0.298, 'SASA':2.368, 'V':  0.023599 },
}

import math

def avg_sd(NUMBERS):
    AVG = sum(NUMBERS)/len(NUMBERS)
    TEM = [pow(NUMBER-AVG, 2) for NUMBER in NUMBERS]
    DEV = sum(TEM)/len(TEM)
    SD = math.sqrt(DEV)
    return (AVG, SD)

# PCPVS: Physicochemical property values
PCPVS = {'H1':[], 'H2':[], 'NCI':[], 'P1':[], 'P2':[], 'SASA':[], 'V':[]}
for AA, PCPS in AAPCPVS.items():
    for PCPN in PCPNS:
        PCPVS[PCPN].append(PCPS[PCPN])

# PCPASDS: Physicochemical property avg and sds
PCPASDS = {}
for PCP, VS in PCPVS.items():
    PCPASDS[PCP] = avg_sd(VS)

# NORMALIZED_AAPCPVS
NORMALIZED_AAPCPVS = {}
for AA, PCPS in AAPCPVS.items():
    NORMALIZED_PCPVS = {}
    for PCP, V in PCPS.items():
        NORMALIZED_PCPVS[PCP] = (V-PCPASDS[PCP][0])/PCPASDS[PCP][1]
    NORMALIZED_AAPCPVS[AA] = NORMALIZED_PCPVS

def pcp_value_of(AA, PCP):
    """Get physicochemical properties value of amino acid."""
    return NORMALIZED_AAPCPVS[AA][PCP];

def pcp_sequence_of(PS, PCP):
    """Make physicochemical properties sequence of protein sequence."""
    PCPS = []
    for I, CH in enumerate(PS):
        PCPS.append(pcp_value_of(CH, PCP))
    # Centralization
    AVG = sum(PCPS)/len(PCPS)
    for I, PCP in enumerate(PCPS):
        PCPS[I] = PCP - AVG
    return PCPS

def ac_values_of(PS, PCP, LAG):
    """Get ac values of protein sequence."""
    AVS = []
    PCPS = pcp_sequence_of(PS, PCP)
    for LG in range(1, LAG+1):
        SUM = 0
        for I in range(len(PCPS)-LG):
            SUM = SUM + PCPS[I]*PCPS[I+LG]
        SUM = SUM / (len(PCPS)-LG)
        AVS.append(SUM)
    return AVS

def all_ac_values_of(PS, LAG):
    """Get all ac values of protein sequence."""
    AAVS = []
    for PCP in PCPS:
        AVS = ac_values_of(PS, PCP, LAG)
        AAVS = AAVS + AVS
    return AAVS

def ac_code_of(PS):
    """Get ac code of protein sequence."""
    AC_Code = all_ac_values_of(PS, 30)
    # Normalizing AC_Code
    # MIN_CODE = min(AC_Code)
    # MAX_CODE = max(AC_Code)
    # AC_Code = [(N-MIN_CODE)*1.0/(MAX_CODE-MIN_CODE) for N in AC_Code]
    return AC_Code

if __name__=="__main__":
    AC = ac_code_of('MKFVYKEEHPFEKRRSEGEKIRKKYPDRVPVIVEKAPKARIGDLDKKKYLVPSDLTVGQFYFLIRKRIHLRAEDALFFFVNNVIPPTSATMGQLYQEHHEEDFFLYIAYSDESVYGL')
    print(AC)

[0.10790803428842154, 0.11173882903839012, -0.14460761970272953, -0.07535489086506163, 0.006237248073411141, 0.0013678539469620598, 0.05270459435279943, 0.1828958100504284, -0.06548284241673244, -0.06356580523179726, -0.0827079689124269, 0.0017860602217885661, 0.13334465990147337, -0.024160265667395202, -0.037172464278242986, -0.06483520719302295, -0.13815087172208335, 0.0897187757953958, -0.040758691200317373, 0.18293023336208467, 0.12094364277218479, 0.09689785077692985, 0.049025209707293, 0.06979361502005867, 0.218261048339127, 0.1893236184497817, 0.018408227449015546, 0.08829965543046875, 0.052992276171663556, -0.12454362437003005, 0.32508896178416097, 0.06277731652196941, -0.17305532936980655, -0.06880909521329583, 0.007179034232072711, -0.0009166973877763069, 0.10437143296785639, 0.21149735698003105, -0.05294867285423665, 0.04381139122214881, 0.17104549428953786, 0.032328074948299, -0.07969681488713316, -0.051078254991167714, 0.07857539050459156, 0.006677363314985386, -0.16450598

### Pseudo amino acid composition

In [12]:
def paac(str_, lambda_=0):
  # str_="ATTRCDEQGGGMFSTQW"
  # lambda_ = 3
  len_=len(str_)
  tt=['A', 'R', 'N', 'D', 'C', 'E', 'Q', 'G', 'H', 'I',  'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']
  A = [0.62,  -0.5, 15]
  R = [-2.53,   3, 101]
  N = [-0.78,  0.2, 58]
  D = [-0.9,    3, 59]
  C = [0.29,    -1, 47]
  E = [-0.74,   3, 73]
  Q = [-0.85,  0.2, 72]
  G =[0.48,    0, 1]
  H = [-0.4,  -0.5, 82]
  I = [1.38, -1.8, 57]
  L = [1.06,  -1.8, 57]
  K = [-1.5,    3, 73]
  M = [0.64,  -1.3, 75]
  F =[1.19, -2.5, 91]
  P = [0.12,     0, 42]
  S = [-0.18, 0.3, 31]
  T = [-0.05, -0.4, 45]
  W = [0.81, -3.4, 130] 
  Y = [0.26,  -2.3, 107]
  V = [1.08, -1.5, 43]
  X = [0, 0, 0]
  H1=[A[0],R[0],N[0],D[0],C[0],E[0],Q[0],G[0],H[0],I[0],L[0],K[0],M[0],F[0],P[0],S[0],T[0],W[0],Y[0],V[0]]
  H2=[A[1],R[1],N[1],D[1],C[1],E[1],Q[1],G[1],H[1],I[1],L[1],K[1],M[1],F[1],P[1],S[1],T[1],W[1],Y[1],V[1]]
  M=[A[2],R[2],N[2],D[2],C[2],E[2],Q[2],G[2],H[2],I[2],L[2],K[2],M[2],F[2],P[2],S[2],T[2],W[2],Y[2],V[2]]
  # Normalization
  mean_H1=np.mean(H1)
  std_H1=np.std(H1)
  H1=(H1-mean_H1)/(std_H1)



  mean_H2=np.mean(H2)
  std_H2=np.std(H2)
  H2=(H2-mean_H2)/(std_H2)

  mean_M=np.mean(M)
  std_M=np.std(M)
  M=(M-mean_M)/(std_M)
  data=np.zeros((1,len_))
  f=np.zeros((1,20))

  for j in range(len_):
      for k in range(20):
          # if strcmp(str(j),tt(k))==1
          if str_[j] == tt[k]:
              # print(j, k)
              data[:,j]=int(k)+1
              f[:,k]=f[:,k]+1
  data = data.astype('int32')
  Theta=np.zeros((lambda_,len_))
  H=np.hstack((H1,H2,M))
  H=H.reshape(3,-1)
  for i in range(lambda_):
      # for j=1:len-i
      for j in range(len_-i):
          if j+i+1<len_:
              Theta[i,j]=np.mean(np.mean((H[:, data[:,j]-1]-H[:, data[:,j+i+1]-1])**2))

  theta=np.zeros((1,lambda_))
  for j in range(lambda_):
      theta[:,j]=np.mean(Theta[j,:(len_-j-1)])

  f=f/len_
  XC=f/(1+0.05*np.sum(theta))
  XC2=(0.05*theta)/(1+0.05*np.sum(theta))

  paac = np.hstack((XC, XC2))
  paac = paac.reshape(-1,).tolist()
  return paac


# 23 dimension paac vector
# paac(seq, 3)

### Amino acid composition

In [13]:
def aac(seq):
  aa_list = ['A', 'R', 'N', 'D', 'C', 'E', 'Q', 'G', 'H', 'I',  'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']
  sum_freq = 0
  for i in range(20):
    sum_freq += seq.count(aa_list[i])

  aa_freq = []
  for i in range(20):
    aa_freq.append(seq.count(aa_list[i])/sum_freq)
  return aa_freq

### Concatenate of protein sequence features

In [14]:
def encode_seq(seq):
  encoding = aac(seq)+ct_code_of(seq)+ld_code_of(seq)
  encoding = np.array(encoding)
  encoding = encoding.reshape(-1, )
  return encoding

### Use universal embedding files

In [37]:
if available_data == 0:
  id2seq_file = 'hp_dict.tsv'
  id2index = {}
  seqs = []
  index = 0
  sid1_index = 0
  sid2_index = 1
  ds_file = 'hp_pairs.tsv'
  label_index = 2
  use_emb = 'ac5_aph.txt'


  # Create line variable as a list of protein sequences with index is the number of protein sequences
  # id2index is a dictionary of protein id and incremental index number 
  for line in open(id2seq_file):
      line = line.strip().split('\t')
      id2index[line[0]] = index
      seqs.append(line[1])
      index += 1

  seq_array = []
  id2_aid = {}
  sid = 0

  seq2t = s2t(use_emb)

  max_data = -1
  limit_data = max_data > 0
  raw_data = []
  skip_head = True
  x = None
  count = 0

  # Create sequence array as a list of protein strings
  for line in tqdm(open(ds_file)):
      if skip_head:
          skip_head = False
          continue
      line = line.rstrip('\n').rstrip('\r').split('\t')
      if id2index.get(line[sid1_index]) is None or id2index.get(line[sid2_index]) is None:
          continue
      if id2_aid.get(line[sid1_index]) is None:
          id2_aid[line[sid1_index]] = sid
          sid += 1
          seq_array.append(seqs[id2index[line[sid1_index]]])
      line[sid1_index] = id2_aid[line[sid1_index]]
      if id2_aid.get(line[sid2_index]) is None:
          id2_aid[line[sid2_index]] = sid
          sid += 1
          seq_array.append(seqs[id2index[line[sid2_index]]])
      line[sid2_index] = id2_aid[line[sid2_index]]
      raw_data.append(line)
      if limit_data:
          count += 1
          if count >= max_data:
              break

  len_m_seq = np.array([len(line.split()) for line in seq_array])
  avg_m_seq = int(np.average(len_m_seq)) + 1
  max_m_seq = max(len_m_seq)
  dim = seq2t.dim

  # seq_tensor is tensor representation of dataset having shape of (number_of_sequences, padding_length, embedding_dim_of_aa)
  # Random for distribution of class labels
  np.random.seed(42)
  np.random.shuffle(raw_data)

  seq_tensor = np.array([seq2t.embed_normalized(line, seq_size) for line in tqdm(seq_array)])
  # seq_tensor = np.array([encode_seq(line) for line in tqdm(seq_array)])

  # Extract index of 1st and 2nd sequences in pairs
  seq_index1 = np.array([line[sid1_index] for line in tqdm(raw_data)])
  seq_index2 = np.array([line[sid2_index] for line in tqdm(raw_data)])

  # Assign labels for pairs of sequences
  class_map = {'0':1,'1':0}
  class_labels = np.zeros((len(raw_data), 2))
  for i in range(len(raw_data)):
      class_labels[i][class_map[raw_data[i][label_index]]] = 1


2916it [00:00, 177656.59it/s]
100%|██████████| 2915/2915 [00:00<00:00, 894330.78it/s]


### Use language model for embedding

In [16]:
if flags_embedding == 1:
    !pip install bio-embeddings[all] # Need to restart runtime for the first run 

    # Choose protein language model for embedder
    from Bio import SeqIO # From Biopython library import SeqIO module to handle sequences when read and write different file formats

    # Chooose language model embedder class from package bio-embeddings -> moduel embed
    # from bio_embeddings.embed import CPCProtEmbedder, ProtTransT5XLU50Embedder, FastTextEmbedder, GloveEmbedder, PLUSRNNEmbedder, ProtTransBertBFDEmbedder, SeqVecEmbedder, UniRepEmbedder, Word2VecEmbedder, ProtTransXLNetUniRef100Embedder
    #   from bio_embeddings.embed import ProtTransBertBFDEmbedder
    from bio_embeddings.embed import ProtTransT5UniRef50Embedder
    embedder = ProtTransT5UniRef50Embedder()

    # Download raw sequences and create a list of sequences
    !wget https://raw.githubusercontent.com/anhvt00/PIPR/master/yeast/preprocessed/protein_preprocessed.txt
    with open('protein_preprocessed.txt') as file:
        sequences = file.readlines()
        sequences = [sequence.rstrip() for sequence in sequences]

    # Install in the case of using A100 for pytorch compatibility\
    A100_status = !nvidia-smi | grep 'A100'
    if A100_status:
        !pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio==0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

    # embeddings = []
    # for sequence in sequences:
    #     embeddings.append(embedder.embed(sequence))

    # # Start embedding 
    # # Embedding in generator form, need to iterate (flexible)
    embeddings = embedder.embed_many(sequences)

    # # Use list function to convert generator to list (true form of dataset)
    embeddings = list(embeddings)

    # # Average pooling in sequence dimension
    # reduced_embeddings = [ProtTransBertBFDEmbedder.reduce_per_protein(e) for e in embeddings]

    # # Padding to create fixed size tensor
    seq_tensor= tf.keras.preprocessing.sequence.pad_sequences(embeddings,  padding='post', dtype='float16', truncating='post', maxlen=seq_size)
    dim = seq_tensor.shape[2]

### Define custom function 

In [17]:
def leaky_relu(x, alpha = .3):
   return tf.keras.backend.maximum(alpha*x, x)

get_custom_objects().update({'leaky_relu': leaky_relu})

### Search for optimal configurations

In [33]:
HP_EPSILON = hp.HParam('epsilon', hp.Discrete([1e-6]))

HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([1e-3]))

HP_FIRST_DENSE = hp.HParam('first_dense', hp.Discrete([100]))

HP_KERNEL_SIZE = hp.HParam('kernel_size', hp.Discrete([3]))

HP_POOLING_KERNEL = hp.HParam('pooling_kernel', hp.Discrete([3]))

HP_CONV_HIDDEN_DIM = hp.HParam('conv_hidden_dim', hp.Discrete([50]))
HP_RNN_HIDDEN_DIM = hp.HParam('rnn_hidden_dim', hp.Discrete([50]))

HP_ACTIVATION = hp.HParam('activation', hp.Discrete(['leaky_relu']))

HP_ACTIVATION_CONV = hp.HParam('activation_conv', hp.Discrete(['linear']))

HP_REGULARIZER = hp.HParam('regularizer', hp.Discrete([0]))

HP_CONV_PADDING = hp.HParam('conv_padding', hp.Discrete(['valid']))

HP_DROPOUT = hp.HParam('dropout', hp.Discrete([2e-1]))

HP_BATCH_SIZE = hp.HParam('batch_size', hp.Discrete([32]))

HP_LEAKY_RELU = hp.HParam('leaky_relu', hp.Discrete([3e-1]))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_EPSILON,HP_LEARNING_RATE,HP_FIRST_DENSE, HP_KERNEL_SIZE, HP_POOLING_KERNEL, HP_CONV_HIDDEN_DIM, HP_RNN_HIDDEN_DIM, HP_ACTIVATION, HP_ACTIVATION_CONV, HP_REGULARIZER, HP_CONV_PADDING, HP_DROPOUT, HP_BATCH_SIZE, HP_LEAKY_RELU],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

### Create dataset from generator

In [19]:
def generator_pair(dataset):
  for index in dataset:
    yield {"seq1": seq_tensor[seq_index1[index]], "seq2": seq_tensor[seq_index2[index]]}, class_labels[index]

def generator_pair_predict(dataset):
  for index in dataset:
    yield {"seq1": seq_tensor[seq_index1[index]], "seq2": seq_tensor[seq_index2[index]]}

# def generator_pair_augment(dataset, augment_data):
#   if augment_data == False:
#     for index in dataset:
#       yield {"seq1": seq_tensor[seq_index1[index]], "seq2": seq_tensor[seq_index2[index]]}, class_labels[index]
#   else:
#     for index in dataset:
#       yield {"seq1": seq_tensor[seq_index1[index]], "seq2": seq_tensor[seq_index2[index]]}, class_labels[index]
#       yield {"seq2": seq_tensor[seq_index1[index]], "seq1": seq_tensor[seq_index2[index]]}, class_labels[index]

### Split the dataset

In [38]:
### k-fold cross-validation
from sklearn.model_selection import KFold, ShuffleSplit
from sklearn.model_selection import train_test_split
kf = KFold(n_splits=5, random_state=42, shuffle=True)
tries = 5
cur = 0
recalls = []
accuracy = []
total = []
total_truth = []
train_test = []
for train, test in kf.split(class_labels):
    # redundant because same position
    # if np.sum(class_labels[train], 0)[0] > 0.8 * len(train) or np.sum(class_labels[train], 0)[0] < 0.2 * len(train):
    #     continue
    train_test.append((train, test))
    cur += 1
    if cur >= tries:
        break


### Define callbacks for monitor

In [29]:
### Define tensorboard callback to optimize resource using of model
logs = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")

tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '20, 29')

### Learning rate schedule for optimization during training
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.,
    patience=10,
    verbose=0,
    mode="auto",
    min_lr=1e-4)

# Schedule early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    verbose=1,
    patience=10,
    mode='auto',
    restore_best_weights=True)


### Define performance metrics

In [22]:
!pip install tensorflow-addons
import tensorflow_addons as tfa
METRICS = [
      # keras.metrics.Accuracy(name='accuracy'),
      # keras.metrics.TruePositives(name='tp'),
      # keras.metrics.FalsePositives(name='fp'),
      # keras.metrics.TrueNegatives(name='tn'),
      # keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      tfa.metrics.MatthewsCorrelationCoefficient(num_classes=2, name='mcc'),
      tfa.metrics.F1Score(num_classes=2, threshold=0.5, name='f1-score'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]



     |████████████████████████████████| 1.1 MB 7.4 MB/s 


### Original architecture PIPR

In [44]:
def build_model(hparams):
    # Input of sequence tensor representations 
    seq_input1 = Input(shape=(seq_size, dim), name='seq1')
    seq_input2 = Input(shape=(seq_size, dim), name='seq2')

    # Define Conv1D and Bi-RNN (GRU/LSTM) use in architecture
    l1=Conv1D(hparams[HP_CONV_HIDDEN_DIM], hparams[HP_KERNEL_SIZE], activation=hparams[HP_ACTIVATION_CONV], padding=hparams[HP_CONV_PADDING])
    r1=Bidirectional(GRU(hparams[HP_RNN_HIDDEN_DIM], return_sequences=True))
    l2=Conv1D(hparams[HP_CONV_HIDDEN_DIM], hparams[HP_KERNEL_SIZE], activation=hparams[HP_ACTIVATION_CONV], padding=hparams[HP_CONV_PADDING])
    r2=Bidirectional(GRU(hparams[HP_RNN_HIDDEN_DIM], return_sequences=True))
    l3=Conv1D(hparams[HP_CONV_HIDDEN_DIM], hparams[HP_KERNEL_SIZE], activation=hparams[HP_ACTIVATION_CONV], padding=hparams[HP_CONV_PADDING])
    r3=Bidirectional(GRU(hparams[HP_RNN_HIDDEN_DIM], return_sequences=True))
    l4=Conv1D(hparams[HP_CONV_HIDDEN_DIM], hparams[HP_KERNEL_SIZE], activation=hparams[HP_ACTIVATION_CONV], padding=hparams[HP_CONV_PADDING])
    r4=Bidirectional(GRU(hparams[HP_RNN_HIDDEN_DIM], return_sequences=True))
    l5=Conv1D(hparams[HP_CONV_HIDDEN_DIM], hparams[HP_KERNEL_SIZE], activation=hparams[HP_ACTIVATION_CONV], padding=hparams[HP_CONV_PADDING])
    r5=Bidirectional(GRU(hparams[HP_RNN_HIDDEN_DIM], return_sequences=True))
    l6=Conv1D(hparams[HP_CONV_HIDDEN_DIM], hparams[HP_KERNEL_SIZE], activation=hparams[HP_ACTIVATION_CONV], padding=hparams[HP_CONV_PADDING])
    
    # Siamese architecture

    ### 1st sibling

    # 1st Block RCNN 
    s1=MaxPooling1D(hparams[HP_POOLING_KERNEL])(l1(seq_input1))
    s1=concatenate([r1(s1), s1])

    # 2nd Block RCNN
    s1=MaxPooling1D(hparams[HP_POOLING_KERNEL])(l2(s1))
    s1=concatenate([r2(s1), s1])

    # 3rd Block RCNN
    s1=MaxPooling1D(hparams[HP_POOLING_KERNEL])(l3(s1))
    s1=concatenate([r3(s1), s1])

    # 4th Block RCNN 
    s1=MaxPooling1D(hparams[HP_POOLING_KERNEL])(l4(s1))
    s1=concatenate([r4(s1), s1])

    # 5th Block RCNN
    s1=MaxPooling1D(hparams[HP_POOLING_KERNEL])(l5(s1))
    s1=concatenate([r5(s1), s1])
    
    # Last convolution
    s1=l6(s1)
    s1=GlobalAveragePooling1D()(s1)

    ### 2nd sibling

    # 1st block RCNN
    s2=MaxPooling1D(hparams[HP_POOLING_KERNEL])(l1(seq_input2))
    s2=concatenate([r1(s2), s2])

    # 2nd block RCNN
    s2=MaxPooling1D(hparams[HP_POOLING_KERNEL])(l2(s2))
    s2=concatenate([r2(s2), s2])

    # 3rd block RCNN
    s2=MaxPooling1D(hparams[HP_POOLING_KERNEL])(l3(s2))
    s2=concatenate([r3(s2), s2])

    # 4th block RCNN
    s2=MaxPooling1D(hparams[HP_POOLING_KERNEL])(l4(s2))
    s2=concatenate([r4(s2), s2])

    # 5th block RCNN
    s2=MaxPooling1D(hparams[HP_POOLING_KERNEL])(l5(s2))
    s2=concatenate([r5(s2), s2])

    # Last convolution
    s2=l6(s2)
    s2=GlobalAveragePooling1D()(s2)

    merged_text = multiply([s1, s2])

    # ## Combine two siblings of siamese architecture
    # abs_sub = tf.keras.layers.Lambda(lambda x: tf.abs(x))(s1-s2)
    # merged_text = concatenate([s1, s2])
    
    # merged = Dense(100, activation='leaky_relu',  kernel_regularizer='l2')(merged_text)
    # merged = Dropout(.2)(merged)
 
    # merged_cos = Dense(100, activation='leaky_relu',  kernel_regularizer='l2')(tf.math.cos(math.pi*merged_text))
    # merged_cos = Dropout(.2)(merged_cos)
    
    # merged_sin = Dense(100, activation='leaky_relu',  kernel_regularizer='l2')(tf.math.sin(math.pi*merged_text))
    # merged_sin = Dropout(.2)(merged_sin)
    
    # merged_text = merged + merged_cos + merged_sin

    #### MLP Part
    # Set initializer
    
    # First dense
    x = Dense(hparams[HP_FIRST_DENSE], activation=hparams[HP_ACTIVATION])(merged_text)
    # x = tf.keras.layers.LeakyReLU(alpha=.3)(x)
    x = Dropout(hparams[HP_DROPOUT])(x)

    # Second dense
    x = Dense(int((hparams[HP_CONV_HIDDEN_DIM]+7)/2), activation=hparams[HP_ACTIVATION])(x)
    # x = tf.keras.layers.LeakyReLU(alpha=.3)(x)
    x = Dropout(hparams[HP_DROPOUT])(x)

    # Last softmax
    main_output = Dense(2, activation='softmax')(x)

    # Combine to form functional model
    merge_model = Model(inputs=[seq_input1, seq_input2], outputs=[main_output])
    return merge_model

### Summary of model architecture

In [30]:
hparams = {
  HP_EPSILON: EPSILON,
  HP_LEARNING_RATE: LEARNING_RATE,
  HP_FIRST_DENSE: 100,
  HP_KERNEL_SIZE: 3,
  HP_POOLING_KERNEL: 3,
  HP_CONV_HIDDEN_DIM: 50,
  HP_RNN_HIDDEN_DIM: 50,
  HP_ACTIVATION: 'leaky_relu',
  HP_ACTIVATION_CONV: 'relu',
  HP_REGULARIZER: 0,
  HP_CONV_PADDING: 'valid',
  HP_DROPOUT: 3e-1,
  HP_BATCH_SIZE: 256,
  HP_LEAKY_RELU: 3e-1
}

model = build_model(hparams)
tf.keras.utils.plot_model(model, to_file='model.png', show_shapes=True)

AttributeError: ignored

### Config Train-test process

In [ ]:
!rm -rf my_best_model_*

In [42]:
def train_test_model(hparams):
  training_time = 1
  num_hit = 0.
  num_total = 0.
  num_pos = 0.
  num_true_pos = 0.
  num_false_pos = 0.
  num_true_neg = 0.
  num_false_neg = 0.
  for train, test in train_test:
      merge_model = None
      merge_model = build_model(hparams)  


      merge_model.compile(optimizer=Adam(learning_rate=hparams[HP_LEARNING_RATE], amsgrad=True, epsilon=hparams[HP_EPSILON]), loss='categorical_crossentropy', metrics=METRICS)
      # Create train
      train_dataset = tf.data.Dataset.from_generator(generator_pair, args=[train], output_types=({"seq1": DTYPE, "seq2": DTYPE}, DTYPE), output_shapes = ({"seq1": (seq_size, dim), "seq2": (seq_size, dim)}, (2,)) )
      train_dataset = train_dataset.shuffle(256).repeat(N_EPOCHS).batch(hparams[HP_BATCH_SIZE])
      train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)


      # Create val
      val_dataset = tf.data.Dataset.from_generator(generator_pair, args=[test], output_types=({"seq1": DTYPE, "seq2": DTYPE}, DTYPE), output_shapes = ({"seq1": (seq_size, dim), "seq2": (seq_size, dim)}, (2,)) )
      val_dataset = val_dataset.batch(hparams[HP_BATCH_SIZE])
      val_dataset = val_dataset.prefetch(tf.data.AUTOTUNE)

      # Save the best model base on val_accuracy
      checkpoint = ModelCheckpoint(filepath=f'my_best_model_{training_time}.hdf5', monitor='val_accuracy',verbose=1, save_best_only=True, mode='max')
      # Fit model
      print(f'==================== Training time  {training_time} =====================')
      
      # Log learning curve to each training time
      history = {}
      history[f'training time {training_time}'] = merge_model.fit(train_dataset, steps_per_epoch=len(train)//hparams[HP_BATCH_SIZE], epochs=N_EPOCHS, validation_data=val_dataset, callbacks=[checkpoint, reduce_lr, early_stopping], shuffle=True)
      # merge_model.fit([seq_tensor[seq_index1[train]], seq_tensor[seq_index2[train]]], class_labels[train], batch_size = hparams[HP_BATCH_SIZE], epochs=N_EPOCHS, validation_data=([seq_tensor[seq_index1[test]], seq_tensor[seq_index2[test]]], class_labels[test]), callbacks=[checkpoint, reduce_lr])

      print(f'==================End training {training_time}========================')
      # # Create test
      # test_dataset = tf.data.Dataset.from_generator(generator_pair, args=[test], output_types=({"seq1": DTYPE, "seq2": DTYPE}, DTYPE), output_shapes = ({"seq1": (seq_size, dim), "seq2": (seq_size, dim)}, (2,)) )
      # test_dataset = test_dataset.batch(hparams[HP_BATCH_SIZE])
      # res = merge_model.evaluate(test_dataset)
      # Create pred
      pred_dataset = tf.data.Dataset.from_generator(generator_pair_predict, args=[test], output_types=({"seq1": DTYPE, "seq2": DTYPE}), output_shapes = ({"seq1": (seq_size, dim), "seq2": (seq_size, dim)}) )
      pred_dataset = pred_dataset.batch(BATCH_SIZE)
      pred_dataset = pred_dataset.prefetch(tf.data.AUTOTUNE)
      pred = merge_model.predict(pred_dataset)

      # Performance metrics
      for i in range(len(class_labels[test])):
          num_total += 1
          if np.argmax(class_labels[test][i]) == np.argmax(pred[i]):
              num_hit += 1
          if class_labels[test][i][0] > 0.:
              num_pos += 1.
              if pred[i][0] > pred[i][1]:
                  num_true_pos += 1
              else:
                  num_false_neg += 1
          else:
              if pred[i][0] > pred[i][1]:
                  num_false_pos += 1
              else:
                  num_true_neg += 1
      accuracy = num_hit / num_total
      prec = num_true_pos / (num_true_pos + num_false_pos)
      recall = num_true_pos / num_pos
      spec = num_true_neg / (num_true_neg + num_false_neg)
      f1 = 2. * prec * recall / (prec + recall)
      # mcc = (num_true_pos * num_true_neg - num_false_pos * num_false_neg) / ((num_true_pos + num_true_neg) * (num_true_pos + num_false_neg) * (num_false_pos + num_true_neg) * (num_false_pos + num_false_neg)) ** 0.5
      mcc = (num_true_pos * num_true_neg - num_false_pos * num_false_neg) / ((num_true_pos + num_false_pos) * (num_true_pos + num_false_neg) * (num_true_neg + num_false_pos) * (num_true_neg + num_false_neg)) ** 0.5
      training_time += 1
      print (f'accuracy: {accuracy}, precision: {prec}, recall: {recall}, specificity: {spec}, mcc: {mcc} ,f1-score: {f1}')
  return accuracy

### Log configurations and accuracy

In [35]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

### Train the neural network through all configurations

In [195]:
# from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
# for i in range(1441):
#   scaler = StandardScaler().fit(seq_tensor[i])

#   # scaler = RobustScaler().fit(X)
#   seq_tensor[i]= scaler.transform(seq_tensor[i])

In [ ]:
# Remove log directory from last run
%%time
!rm -rf ./logs
session_num = 0
seq_size=seq_tensor.shape[1]
dim=seq_tensor.shape[2]
for epsilon in HP_EPSILON.domain.values:
  for learning_rate in HP_LEARNING_RATE.domain.values:
    for first_dense in HP_FIRST_DENSE.domain.values:
      for kernel_size in HP_KERNEL_SIZE.domain.values:
        for pooling_kernel in HP_POOLING_KERNEL.domain.values:
          for conv_hidden_dim in HP_CONV_HIDDEN_DIM.domain.values:
            for rnn_hidden_dim in HP_RNN_HIDDEN_DIM.domain.values:
              for activation in HP_ACTIVATION.domain.values:
                for activation_conv in HP_ACTIVATION_CONV.domain.values:
                  for regularizer in HP_REGULARIZER.domain.values:
                    for conv_padding in HP_CONV_PADDING.domain.values:
                      for dropout in HP_DROPOUT.domain.values:
                        for batch_size in HP_BATCH_SIZE.domain.values:
                          for leaky_relu in HP_LEAKY_RELU.domain.values:
                            hparams = {
                                HP_EPSILON: epsilon,
                                HP_LEARNING_RATE: learning_rate,
                                HP_FIRST_DENSE: first_dense,
                                HP_KERNEL_SIZE: kernel_size,
                                HP_POOLING_KERNEL: pooling_kernel,
                                HP_CONV_HIDDEN_DIM: conv_hidden_dim,
                                HP_RNN_HIDDEN_DIM: rnn_hidden_dim,
                                HP_ACTIVATION: activation,
                                HP_ACTIVATION_CONV: activation_conv,
                                HP_REGULARIZER: regularizer,
                                HP_CONV_PADDING: conv_padding,
                                HP_DROPOUT: dropout,
                                HP_BATCH_SIZE: batch_size,
                                HP_LEAKY_RELU: leaky_relu
                            }
                            run_name = "run-%d" % session_num
                            print('--- Starting trial: %s' % run_name)
                            print({h.name: hparams[h] for h in hparams})
                            run('logs/hparam_tuning/' + run_name, hparams)
                            session_num += 1


--- Starting trial: run-0
{'epsilon': 1e-06, 'learning_rate': 0.001, 'first_dense': 100, 'kernel_size': 3, 'pooling_kernel': 3, 'conv_hidden_dim': 50, 'rnn_hidden_dim': 50, 'activation': 'leaky_relu', 'activation_conv': 'linear', 'regularizer': 0, 'conv_padding': 'valid', 'dropout': 0.2, 'batch_size': 32, 'leaky_relu': 0.3}
==================== Training time  1 =====================
Epoch 1/80
72/72 [==============================] - ETA: 0s - loss: 0.6703 - accuracy: 0.6072 - precision: 0.6072 - recall: 0.6072 - mcc: 0.2190 - f1-score: 0.6007 - auc: 0.6413 - prc: 0.6239
Epoch 00001: val_accuracy improved from -inf to 0.61921, saving model to my_best_model_1.hdf5


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


72/72 [==============================] - 72s 263ms/step - loss: 0.6703 - accuracy: 0.6072 - precision: 0.6072 - recall: 0.6072 - mcc: 0.2190 - f1-score: 0.6007 - auc: 0.6413 - prc: 0.6239 - val_loss: 0.6470 - val_accuracy: 0.6192 - val_precision: 0.6192 - val_recall: 0.6192 - val_mcc: 0.2367 - val_f1-score: 0.6173 - val_auc: 0.6789 - val_prc: 0.6678 - lr: 0.0010
Epoch 2/80
72/72 [==============================] - ETA: 0s - loss: 0.6630 - accuracy: 0.6241 - precision: 0.6241 - recall: 0.6241 - mcc: 0.2525 - f1-score: 0.6196 - auc: 0.6501 - prc: 0.6359
Epoch 00002: val_accuracy improved from 0.61921 to 0.63293, saving model to my_best_model_1.hdf5
72/72 [==============================] - 10s 144ms/step - loss: 0.6630 - accuracy: 0.6241 - precision: 0.6241 - recall: 0.6241 - mcc: 0.2525 - f1-score: 0.6196 - auc: 0.6501 - prc: 0.6359 - val_loss: 0.6508 - val_accuracy: 0.6329 - val_precision: 0.6329 - val_recall: 0.6329 - val_mcc: 0.2898 - val_f1-score: 0.6253 - val_auc: 0.6675 - val_prc: 0

### Extract last hidden layer for input of LGBM

In [197]:
################################Intermediate Layer prediction (Abstraction features extraction)######################################
model = tf.keras.models.load_model('/content/my_best_model_1.hdf5')
intermediate_layer_model = Model(inputs=model.input,outputs=model.get_layer(model.layers[-3].name).output)

# Use intermediate layer to transform pairs matrix
intermediate_output_p1 = intermediate_layer_model.predict([seq_tensor[seq_index1],seq_tensor[seq_index2]])  


p_merge=pd.DataFrame(intermediate_output_p1)    
Trainlabels = np.zeros((len(class_labels),))
for i in range(len(class_labels)):
  if np.allclose(np.array([1., 0.]), class_labels[i]):
    Trainlabels[i] = 1.
# create dataframe use transformed pairs matrix outputs and labels
X_train_feat=pd.concat((p_merge,pd.DataFrame(pd.DataFrame(Trainlabels))),axis=1,ignore_index=True)

# write to file dataframe of transformed pairs matrix and labels
X_train_feat.to_csv('X_train.csv',header=False, index=False)

# read dataframe of transformed pairs matrix and labels
Train=pd.read_csv("X_train.csv",header=None)
Train=Train.sample(frac=1)

X=Train.iloc[:,0:28].values
y=Train.iloc[:,28:].values

extracted_df=X_train_feat

scaler=RobustScaler()
# scaler=MinMaxScaler()
# scaler = StandardScaler()
X=scaler.fit_transform(X)

### Cross-validation with PIPR-LGBM

In [199]:
##################################### Five-fold Cross-Validation ##########################################################
kf=KFold(n_splits=5, random_state=42, shuffle=True)


accuracy = []
specificity = []
sensitivity = []
precision=[]
recall=[]
m_coef=[]

auc_list=[]
xgb_fpr_list=[]
xgb_tpr_list=[]
o=0
max_accuracy=float("-inf")
xgb_fpr=None
xgb_tpr=None

y = y.reshape(-1, )
training_time = 1
for train, test in kf.split(X,y):
    o=o+1
    model_=LGBMClassifier(learning_rate=.2, gamma=0, max_depth=10, n_estimators=1000)
    # model_=XGBClassifier(n_estimators=200)
    print(f"========================== START TRAINING TIME {training_time} ===========================")
    hist=model_.fit(X[train], y[train],eval_set=[(X[test], y[test])],verbose=False)
    # hist = lgb.train(param, train_data, num_round, valid_sets=[validation_data])
    y_score=model_.predict_proba(X[test])
    y_test=tf.keras.utils.to_categorical(y[test]) 
    
    # fpr, tpr, _ = roc_curve(y_test[:,0].ravel(), y_score[:,0].ravel())
    auc = metrics.roc_auc_score(y_test, y_score)
    auc_list.append(auc)
    coef=matthews_corrcoef(y_test.argmax(axis=1), y_score.argmax(axis=1), sample_weight=None)
    m_coef.append(coef)
    
    cm1=confusion_matrix(y_test.argmax(axis=1), y_score.argmax(axis=1))
    acc = (cm1[0,0]+cm1[1,1])/(cm1[0,0]+cm1[0,1]+cm1[1,0]+cm1[1,1])
    spec= (cm1[0,0])/(cm1[0,0]+cm1[0,1])
    sens = (cm1[1,1])/(cm1[1,0]+cm1[1,1])
    prec=cm1[1,1]/(cm1[1,1]+cm1[0,1])
    rec=cm1[1,1]/(cm1[1,1]+cm1[1,0])
    sensitivity.append(sens)
    specificity.append(spec)
    accuracy.append(acc)
    precision.append(prec)
    recall.append(rec)
    # xgb_fpr_list.append(fpr)
    # xgb_tpr_list.append(tpr)
    print(f"========================== PERFOMANCE METRICS AT TRAINING TIME {training_time} ===========================")
    print(f"accuracy: {acc}, specificity: {spec}, sensitivity: {sens}, precision= {prec}, recall: {rec} ")
    # if max_accuracy<acc:
    #     max_accuracy=acc
    #     xgb_fpr=fpr
    #     xgb_tpr=tpr
    training_time += 1
        
print("====================================== END CROSS VALIDATION ===============================\n")
xgb_fpr=pd.DataFrame(xgb_fpr)
xgb_tpr=pd.DataFrame(xgb_tpr)

xgb_fpr.to_csv('fprdnn_xgb.csv',header=False, index=False)
xgb_tpr.to_csv('tprdnn_xgb.csv',header=False, index=False)   
 
mean_acc=np.mean(accuracy)
std_acc=np.std(accuracy)
var_acc=np.var(accuracy)
print("Accuracy:"+str(mean_acc)+" Â± "+str(std_acc))
print("Accuracy_Var:"+str(mean_acc)+" Â± "+str(var_acc))
mean_spec=np.mean(specificity)
std_spec=np.std(specificity)
print("Specificity:"+str(mean_spec)+" Â± "+str(std_spec))
mean_sens=np.mean(sensitivity)
std_sens=np.std(sensitivity)
print("Sensitivity:"+str(mean_sens)+" Â± "+str(std_sens))
mean_prec=np.mean(precision)
std_prec=np.std(precision)
print("Precison:"+str(mean_prec)+" Â± "+str(std_prec))
mean_rec=np.mean(recall)
std_rec=np.std(recall)
print("Recall:"+str(mean_rec)+" Â± "+str(std_rec))
mean_coef=np.mean(m_coef)
std_coef=np.std(m_coef)
print("MCC:"+str(mean_coef)+" Â± "+str(std_coef))

print("AUC:"+str(np.mean(auc_list)))



========================== START TRAINING TIME 1 ===========================
========================== PERFOMANCE METRICS AT TRAINING TIME 1 ===========================
accuracy: 0.9725557461406518, specificity: 0.9640522875816994, sensitivity: 0.9819494584837545, precision= 0.9611307420494699, recall: 0.9819494584837545 
========================== START TRAINING TIME 2 ===========================
========================== PERFOMANCE METRICS AT TRAINING TIME 2 ===========================
accuracy: 0.9536878216123499, specificity: 0.9560439560439561, sensitivity: 0.9516129032258065, precision= 0.9609120521172638, recall: 0.9516129032258065 
========================== START TRAINING TIME 3 ===========================
========================== PERFOMANCE METRICS AT TRAINING TIME 3 ===========================
accuracy: 0.9725557461406518, specificity: 0.9715302491103203, sensitivity: 0.9735099337748344, precision= 0.9735099337748344, recall: 0.9735099337748344 
=========================

### t-sne visualization of last hiden layer extracted by PIPR

In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
def TSNE_extracted():
    
    pos=extracted_df[extracted_df.iloc[:,-1]==1]
    neg=extracted_df[extracted_df.iloc[:,-1]==0]
    X_feat=pd.concat([pos,neg])
    X_feat=X_feat.iloc[:,:-1]
    t=TSNE(n_components=2).fit_transform(X_feat)
    pos_t=t[:int(len(t)/2),:]
    neg_t=t[int(len(t)/2):,:]
    plt.scatter(pos_t[:,0],pos_t[:,1],label="Positive",s=4)
    plt.scatter(neg_t[:,0],neg_t[:,1],label="Negative",s=4)
    plt.legend()
    plt.show()

TSNE_extracted()

### Tensorboard monitor

In [ ]:
# %reload_ext tensorboard
# %tensorboard --logdir=/content/logs